In [1]:
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.local_env import VoronoiNN
from pymatgen.core import Structure
import numpy as np
import pandas as pd

from ase.build import bulk

In [2]:
def get_stats(property_list, property_str):
    return  [f"{property_str}_std",f"{property_str}_mean",f"{property_str}_min",f"{property_str}_max"],\
            [np.std(property_list), np.mean(property_list), np.min(property_list), np.max(property_list)]
            
def VoronoiSiteFeaturiser(structure, site):
    structure = AseAtomsAdaptor().get_structure(structure)
    coord_no = VoronoiNN().get_cn(structure = structure, n = site)
    site_info_dict = VoronoiNN().get_voronoi_polyhedra(structure, site)
    volumes = [site_info_dict[polyhedra]["volume"] for polyhedra in list(site_info_dict.keys())]
    vertices = [site_info_dict[polyhedra]["n_verts"] for polyhedra in list(site_info_dict.keys())]
    distances = [site_info_dict[polyhedra]["face_dist"] for polyhedra in list(site_info_dict.keys())]
    areas = [site_info_dict[polyhedra]["area"] for polyhedra in list(site_info_dict.keys())]
    
    total_area = np.sum(areas)
    total_volume = np.sum(volumes)
    
    df_str_list = ["VorNN_CoordNo", "VorNN_tot_vol", "VorNN_tot_area"]
    df_prop_list = [coord_no, total_volume, total_area]
    
    data_str_list = ["volumes", "vertices", "areas", "distances"]

    for i, value_list in enumerate([volumes, vertices, areas, distances]):
        property_str_list, property_stats_list = get_stats(value_list, f"VorNN_{data_str_list[i]}")
        df_str_list += property_str_list
        df_prop_list += property_stats_list
    
    return df_str_list, df_prop_list

In [9]:
import pandas as pd
import numpy as np
from pymatgen.analysis.local_env import VoronoiNN
from pymatgen.core.structure import Structure
from pymatgen.io.ase import AseAtomsAdaptor

def get_stats(property_list, property_str):
    return {
        f"{property_str}_std": np.std(property_list),
        f"{property_str}_mean": np.mean(property_list),
        f"{property_str}_min": np.min(property_list),
        f"{property_str}_max": np.max(property_list)
    }

def VoronoiSiteFeaturiser(structure, site):
    structure = AseAtomsAdaptor().get_structure(structure)
    coord_no = VoronoiNN().get_cn(structure=structure, n=site)
    site_info_dict = VoronoiNN().get_voronoi_polyhedra(structure, site)
    volumes = [site_info_dict[polyhedra]["volume"] for polyhedra in site_info_dict]
    vertices = [site_info_dict[polyhedra]["n_verts"] for polyhedra in site_info_dict]
    distances = [site_info_dict[polyhedra]["face_dist"] for polyhedra in site_info_dict]
    areas = [site_info_dict[polyhedra]["area"] for polyhedra in site_info_dict]

    total_area = np.sum(areas)
    total_volume = np.sum(volumes)

    data = {
        "VorNN_CoordNo": coord_no,
        "VorNN_tot_vol": total_volume,
        "VorNN_tot_area": total_area
    }

    data_str_list = ["volumes", "vertices", "areas", "distances"]

    for i, value_list in enumerate([volumes, vertices, areas, distances]):
        stats = get_stats(value_list, f"VorNN_{data_str_list[i]}")
        data.update(stats)

    df = pd.DataFrame(data, index=[site])
    return df


In [4]:
struct = bulk("Fe", cubic=True)

In [10]:
struct = bulk("Fe", cubic=True)
df = VoronoiSiteFeaturiser(struct, 1)

,VorNN_CoordNo,VorNN_tot_vol,VorNN_tot_area,VorNN_volumes_std,VorNN_volumes_mean,VorNN_volumes_min,VorNN_volumes_max,VorNN_vertices_std,VorNN_vertices_mean,VorNN_vertices_min,VorNN_vertices_max,VorNN_areas_std,VorNN_areas_mean,VorNN_areas_min,VorNN_areas_max,VorNN_distances_std,VorNN_distances_mean,VorNN_distances_min,VorNN_distances_max
1,14,11.819951,27.577769,0.304654,0.844282,0.492498,1.10812,0.989743,5.142857,4,6,0.814261,1.969841,1.029612,2.675012,0.095141,1.325141,1.242746,1.435
